In [33]:
import networkx as nx

In [69]:
#Read the netlist and sort it in topological order
filename = 'c17.net'
with open(filename,'r') as f:
    rawcircuit = f.read().splitlines()
    
# raw_circuit = [' '.join(rawcircuit.split()) for x in rawcircuit]
# data = [' '.join(x.split()) for x in circ]
# raw_circuit[1].split()[1]

In [70]:
#for not gate converting not to nands
raw_circuit = []
for line in rawcircuit:
    line = line.split()
#     print(line[1])
    if line[1] == 'inv':
#         line[1] = 'nand2'
        line.insert(3,line[2])
    line = ' '.join(line)
    raw_circuit.append(line)
    
raw_circuit

['g51 nand2 n_3 n_0 N22',
 'g52 nand2 n_3 n_2 N23',
 'g53 nand2 n_1 N2 n_3',
 'g54 nand2 n_1 N7 n_2',
 'g55 nand2 N1 N3 n_0',
 'g56 nand2 N3 N6 n_1']

In [71]:
n = len(raw_circuit)
edge = []
for i in range(n):
    buf = []
    line = raw_circuit[i].split()
    buf.append((line[2],line[4]))
    buf.append((line[3],line[4]))
    edge.extend(buf)
    
edge

[('n_3', 'N22'),
 ('n_0', 'N22'),
 ('n_3', 'N23'),
 ('n_2', 'N23'),
 ('n_1', 'n_3'),
 ('N2', 'n_3'),
 ('n_1', 'n_2'),
 ('N7', 'n_2'),
 ('N1', 'n_0'),
 ('N3', 'n_0'),
 ('N3', 'n_1'),
 ('N6', 'n_1')]

In [72]:
def node_finder(a,b):
    aset = set(a)
    bset = set(b)
    
    return aset.intersection(bset)


def pin_namer(data):
    i_pins_f = []
    o_pins_f = []
    for i in range(len(data)):
        line = data[i].split()
        i_pins_f.append(line[2])
        i_pins_f.append(line[3])
        o_pins_f.append(line[4])
        
    i_pins_f = set(i_pins_f)
    
    nodes = node_finder(i_pins_f,o_pins_f)
    
#     o_pins = set(o_pins_f).symmetric_difference(list(nodes))
    
    i_pins = i_pins_f.symmetric_difference(list(nodes))
    
    return list(o_pins_f),list(i_pins)

pin_namer(raw_circuit)[1]

['N7', 'N3', 'N6', 'N2', 'N1']

In [73]:
namegate = {}
input_pins = pin_namer(raw_circuit)[1]
output_pins = pin_namer(raw_circuit)[0]
for i in range(n):
    line = raw_circuit[i].split()
    namegate.update({output_pins[i]:line[1]})

for pin in input_pins:
    namegate.update({pin:'pi'})
    
namegate

{'N22': 'nand2',
 'N23': 'nand2',
 'n_3': 'nand2',
 'n_2': 'nand2',
 'n_0': 'nand2',
 'n_1': 'nand2',
 'N7': 'pi',
 'N3': 'pi',
 'N6': 'pi',
 'N2': 'pi',
 'N1': 'pi'}

In [97]:
#Topological order Done!!!
#S&P works too!

g = nx.DiGraph()

g.add_edges_from(edge)

nx.set_node_attributes(g,namegate,name="gateType")

print(g.nodes(data=True))
nl = list(nx.topological_sort(g))
print()
print('Nodes in topological order',nl)


[('n_3', {'gateType': 'nand2'}), ('N22', {'gateType': 'nand2'}), ('n_0', {'gateType': 'nand2'}), ('N23', {'gateType': 'nand2'}), ('n_2', {'gateType': 'nand2'}), ('n_1', {'gateType': 'nand2'}), ('N2', {'gateType': 'pi'}), ('N7', {'gateType': 'pi'}), ('N1', {'gateType': 'pi'}), ('N3', {'gateType': 'pi'}), ('N6', {'gateType': 'pi'})]

Nodes in topological order ['N2', 'N7', 'N1', 'N3', 'N6', 'n_0', 'n_1', 'n_3', 'n_2', 'N22', 'N23']


In [99]:
g.nodes['n_3']

{'gateType': 'nand2'}

In [75]:

list(g.predecessors('n_1'))

['N3', 'N6']

In [76]:
file = 'c17.inputs'
with open(file,'r') as f:
    raw_inputs = f.read().splitlines()
raw_inputs

['N1 N2 N3 N6 N7 ',
 '0 1 0 0 0',
 '0 0 1 0 0',
 '1 0 0 0 0',
 '0 0 1 1 1',
 '1 1 1 1 1',
 '1 1 1 0 0',
 '1 1 1 1 0',
 '1 1 0 0 0',
 '0 1 1 0 1',
 '0 0 1 1 0']

In [77]:
value = {}

for node in nl:
    try:
        value.update({node:raw_inputs[2].split()[raw_inputs[0].split().index(node)]})
    except ValueError:
        pass
value

{'N2': '0', 'N7': '0', 'N1': '0', 'N3': '1', 'N6': '0'}

In [78]:
def AND (n1, n2):
    if int(n1) == 1 and int(n2) == 1:
        return '1'
    else:
        return '0'
    
def NAND (n1, n2):
 
    if int(n1) == 1 and int(n2) == 1:
        return '0'
    else:
        return '1'
    
def OR(n1, n2):
    if int(n1) == 1 or int(n2) == 1:
        return '1'
    else:
        return '0'

def NOR(n1, n2):
    if int(n1) == 1 or int(n2) == 1:
        return '0'
    else:
        return '1'
    
def XOR (n1, n2):
    if int(n1) != int(n2):
        return '1'
    else:
        return '0'
    
def XOR (n1, n2):
    if int(n1) != int(n2):
        return '0'
    else:
        return '1'
    

In [79]:
nl

['N2', 'N7', 'N1', 'N3', 'N6', 'n_0', 'n_1', 'n_3', 'n_2', 'N22', 'N23']

In [80]:
value_t = {}
for node in nl:
    if g.nodes[node]['gateType'] == 'pi':
        pass
#         value_t[node] = value[node]     
    elif g.nodes[node]['gateType'] == 'nand2':
        nands = NAND(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
        value.update({node:nands})
    elif g.nodes[node]['gateType'] == 'and2':
        ands = AND(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
        value.update({node:ands})
    elif g.nodes[node]['gateType'] == 'or2':
        ors = OR(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
        value.update({node:ors})
    elif g.nodes[node]['gateType'] == 'nor2':
        nors = NOR(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
        value.update({node:nors})
    elif g.nodes[node]['gateType'] == 'xor2':
        xors = XOR(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
        value.update({node:xors})
    elif g.nodes[node]['gateType'] == 'xnor2':
        xnors = XNOR(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
        value.update({node:xnors})
    elif g.nodes[node]['gateType'] == 'inv':
        nots = NAND(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[0]])
        value.update({node:nots})
        
value

{'N2': '0',
 'N7': '0',
 'N1': '0',
 'N3': '1',
 'N6': '0',
 'n_0': '1',
 'n_1': '1',
 'n_3': '1',
 'n_2': '1',
 'N22': '0',
 'N23': '0'}

In [81]:
myKeys = list(value.keys())
myKeys.sort()
sorted_dict = {i: value[i] for i in myKeys}
 
sorted_dict

{'N1': '0',
 'N2': '0',
 'N22': '0',
 'N23': '0',
 'N3': '1',
 'N6': '0',
 'N7': '0',
 'n_0': '1',
 'n_1': '1',
 'n_2': '1',
 'n_3': '1'}

In [82]:
raw_circuit

['g51 nand2 n_3 n_0 N22',
 'g52 nand2 n_3 n_2 N23',
 'g53 nand2 n_1 N2 n_3',
 'g54 nand2 n_1 N7 n_2',
 'g55 nand2 N1 N3 n_0',
 'g56 nand2 N3 N6 n_1']

In [83]:
nl

['N2', 'N7', 'N1', 'N3', 'N6', 'n_0', 'n_1', 'n_3', 'n_2', 'N22', 'N23']

In [90]:
#load input data
qvalue = {}

for node in nl:
    try:
        qvalue.update({node:raw_inputs[4].split()[raw_inputs[0].split().index(node)]})
    except ValueError:
        pass
qvalue

{'N2': '0', 'N7': '1', 'N1': '0', 'N3': '1', 'N6': '1'}

In [91]:
nl
input_list = pin_namer(raw_circuit)[1]
input_list

['N7', 'N3', 'N6', 'N2', 'N1']

In [92]:
def update(dic1,dic2):
    up_dic = {}
    for key in dic1:
        if dic1[key]!=dic2[key]:
            up_dic[key] = dic2[key]
    return up_dic

In [93]:
dic1 = {'a':1,'b':2,'c':3}
dic2 = {'a':3,'b':2,'c':4}

update(dic1,dic2)

{'a': 3, 'c': 4}

In [94]:
# Intializing the que
input_pins = raw_inputs[0].split()

# Intializing variables
value = {}
prev={}
new = {}
outpus = []

# Simulation
for i in range(1,n):
#   load the i'th input
    value.update(loader(raw_inputs,i))
    
#   Is the circuit running for first time? Yes the proceed or go to if block
    if i!=1:
        '''
        Load the previous input and use the changer() that returns a 
        dictionary that keys and values which are changed
        now add this keys to the que and clear the dictionary
        '''
        prev.update(loader(raw_inputs,i-1))
        new.update(changer(prev,value))
        input_pins.extend(list(new.keys()))
        new = {} 

#   Run a infinte loop till the queue is empty
    while True:
        '''
        Using try-except block because when the circuit runs for the first time the
        dictionary values only contains the inputs and no other nodes,and thus this 
        gives a key error in below code
        ''' 
        try: 
            '''Pop the first node and push the succsesor'''
            node = input_pins.pop(0)
            input_pins.extend(list(g.successors(node)))
            
            if len(input_pins)!=0:
                if g.nodes[node]['gateType'] == 'pi':
                    pass
                elif g.nodes[node]['gateType'] == 'nand2':
                    nands = NAND(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
                    value.update({node:nands})
                elif g.nodes[node]['gateType'] == 'and2':
                    ands = AND(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
                    value.update({node:ands})
                elif g.nodes[node]['gateType'] == 'or2':
                    ors = OR(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
                    value.update({node:ors})
                elif g.nodes[node]['gateType'] == 'nor2':
                    nors = NOR(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
                    value.update({node:nors})
                elif g.nodes[node]['gateType'] == 'xor2':
                    xors = XOR(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
                    value.update({node:xors})
                elif g.nodes[node]['gateType'] == 'xnor2':
                    xnors = XNOR(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[1]])
                    value.update({node:xnors})
                elif g.nodes[node]['gateType'] == 'inv':
                    nots = NAND(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[0]])
                    value.update({node:nots})
                elif g.nodes[node]['gateType'] == 'buff':
                    buffs = AND(value[list(g.predecessors(node))[0]],value[list(g.predecessors(node))[0]])
                    value.update({node:buffs})
                
            else:
                break
        except KeyError:
            pass
#   Sort the dictionary alphabetically
    value = sort_d(value)  
    outpus.append(list(value.values()))
    

['N7', 'N3', 'N6', 'N2', 'N1']


KeyError: 'n_1'

In [89]:
qqvalue

{'N7': '0',
 'N3': '1',
 'N6': '0',
 'N2': '0',
 'N1': '0',
 'n_2': '1',
 'n_0': '1',
 'n_1': '1',
 'n_3': '1',
 'N23': '0',
 'N22': '0'}

In [228]:
qqvalue == value

True